In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType
import requests
df = spark.read.table("ecommerce.silver.order_base_table")
df = df.withColumn("gross_amount",F.col("quantity")*F.col("unit_price"))
df = df.withColumn("discount_amount",F.round(F.col("gross_amount") * F.col("discount_pct"),2))
df = df.withColumn("sale_amount",F.round(F.col("gross_amount") - F.col("discount_amount") + F.col("tax_amount"),2))
df = df.withColumn("date_id",F.regexp_replace(F.col("dt"),"-","").cast("string"))
df = df.withColumn("coupon_applied",F.when(F.col("coupon_code").isNotNull(),1).otherwise(0))
API_KEY = "cdba2714292e3d0dba3a81c5"
url = f"https://v6.exchangerate-api.com/v6/{API_KEY}/latest/USD"
rates = requests.get(url).json()["conversion_rates"]
rates_to_usd = {cur: 1/value for cur, value in rates.items()}

def convert_to_usd(currency, amount):
    if currency in rates_to_usd:
        return amount * rates_to_usd[currency]
    return None

convert_udf = udf(convert_to_usd, DoubleType())
df = df.withColumn(
    "unit_price_usd",
    convert_udf(col("unit_price_currency"), col("unit_price"))
)

df.write.format("delta").mode("overwrite").saveAsTable("ecommerce.gold.order_base_table")